In [ ]:
from bs4 import BeautifulSoup as soup  # HTML data structure
from urllib.request import urlretrieve, urlopen as uReq  # Web client
from time import time
import os
from multiprocessing import Process, Pool # SubProcessing
import shutil # directory manipulation
import pandas as pd

In [ ]:
def saveImages(car_id, car_soup):
    path = 'images/{}'.format(car_id)
    
    # create specific folder
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.mkdir(path)
    
    # image processing
    images = car_soup.findAll('div', {'class': 'thumbnail-image'})
    for ind, image in enumerate(images):
        img_src = image.find('img')['src']
        urlretrieve(img_src, "{}/{}.jpg".format(path, ind))

In [ ]:
def saveCSV(car_url, car_data_dict, car_soup):
    # FEATURE: car ID
    car_id = car_url.split('/')[5]
    car_data_dict['ID'] = car_id

    # FEATURE: car Price
    gel_price, usd_price = car_soup.findAll('span', {'class': 'car-price'})
    usd_price = usd_price.text
    if usd_price != 'Price negotiable':
        usd_price = usd_price.split()[0]
        usd_price = usd_price.replace(',', '')
        car_data_dict['Price'] = usd_price
    else:
        car_data_dict['Price'] = 'Price negotiable'

    # FEATURE: car customs
    custom_soup = car_soup.find('div', {'class': 'levy'})
    if custom_soup:
        custom = custom_soup.findAll('span')[1].findAll('span')[1].text
        custom = custom.split()[0]
        custom = custom.replace(',', '')
        car_data_dict['Customs'] = custom
    else:
        car_data_dict['Customs'] = ''

    # FEATURE: Data from <tr>-s
    car_data_table = car_soup.findAll('tr')
    for data in car_data_table:
        th1, th2 = data.findAll('th')

        # th1
        th1_divs = th1.findAll('div')
        if len(th1_divs) == 2:
            th1_key, th1_value = map(
                lambda x: x.text.strip(), th1_divs)
            if th1_key in car_data_dict:
                car_data_dict[th1_key] = th1_value

        # th2
        th2_divs = th2.findAll('div')
        if len(th2_divs) == 2:
            th2_key, th2_value = th2_divs
            th2_key = th2_key.text.strip()
            if 'class' in th2_value.i.attrs and 'fa-check' in th2_value.i['class']:
                res = '1'
            else:
                res = '0'

            th2_value = res
            if th2_key in car_data_dict:
                car_data_dict[th2_key] = th2_value

In [ ]:
def _log(ind, start, total, PID):
    now = time()
    ind += 1
    elapsed = int(now-start)
    if elapsed == 0:
        elapsed = 1 
    mean = (elapsed/ind)
    estimated = (total-ind)//mean
    print('Thread {}: Cars {}/{} -- Time Elapsed {}s -- Time Left {}s'.format(PID, ind, total, elapsed, estimated))
        

In [ ]:
def scrapLink(data,):
    start = time()
    
    # data from argument
    PID, links, headers = data
    
    result = []
    for ind, car_url in enumerate(links):
        _log(ind, start, len(links), PID)
        try:
            # go to each car's url
            uClient = uReq(car_url)
            car_soup = soup(uClient.read(), "html.parser")
            uClient.close()

            # DATA
            car_data_dict = {header: '' for header in headers}

            saveCSV(car_url, car_data_dict, car_soup)
            saveImages(car_data_dict['ID'], car_soup)
            
            result.append(car_data_dict)
            if (ind > 10):
                return result
        except:
            print("Error while processing.. {}".format(ind))
    return result

In [ ]:
# read links
linksFile = open('SabaLinks.txt', 'r')
links = [line.rstrip() for line in linksFile]
linksFile.close()

# open folder for writing result
folder = 'data'
if os.path.isdir(folder):
    shutil.rmtree(folder)
os.mkdir(folder)
os.chdir(folder)

# create images folder
os.mkdir('images')

headers = ['ID', 'Manufacturer', 'Model', 'Category', 'Mileage', 'Gear box type', 'Doors',
                'Wheel', 'Color', 'Interior color', 'VIN', 'Leather interior', 'Price', 'Customs']

In [ ]:
# total processes
procces_count = 10

# data for each process
ProcessDataArray = []

# links count for each process
chunks_size = len(links) // procces_count
    
# divide data into chunks
for i in range(procces_count): 
    chunk = links[chunks_size * i: chunks_size * (i + 1)]
    if i == procces_count - 1:
        chunk = links[chunks_size * i:]
    # data for one process
    processData = (i, chunk, headers)
    
    ProcessDataArray.append(processData) 

In [ ]:
# create pool of processes
p = Pool(procces_count)

# map each process to function "scraplink" with and itterate over argument ProcessDataArray
results = p.map(scrapLink, ProcessDataArray)

# wait for all processes to stop working
p.terminate()
p.join()

In [ ]:
# concatenate all data into one dataframe
final_result = pd.DataFrame(columns = headers)
for result in results:
    final_result = final_result.append(pd.DataFrame(result))

In [ ]:
# write features dataframe into csv
file_name = 'cars.csv'
final_result.to_csv(file_name, index=False)